In [39]:
import numpy as np
import tespy
from matplotlib import pyplot as plt

from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.components import CombustionChamber # adiabatic ?
from tespy.components import (HeatExchanger, Condenser, Desuperheater)
from tespy.components import (Drum, Merge, Separator, Splitter)
from tespy.components import (Pump, Turbine)
from tespy.connections import Connection, Bus
from tespy.tools import ExergyAnalysis
from CoolProp.CoolProp import PropsSI as cpsi

In [52]:
# FLUIDS
wf = 'Water'
fld_wf = {wf:1}

abgas = 'CarbonDioxide'
fld_abgas = {abgas:1}

#gas = 'Methane' # 30 bar and 15°
#fld_gas = {'CH4':1}

#air = 'Air' # 1 bar and 15°
#fld_air = {'N2': 0.79, 'O2': 0.21}


# NETWORK
dampferzeuger = Network(T_unit = 'C', p_unit = 'bar', h_unit = 'kJ / kg', m_unit = 't / h')


# COMPONENTS
# Sources
src_water = Source('Water_source')
src_abgas = Source('Abgas_source')
# Sinks
snk_high = Sink('High_sink')
snk_mid = Sink('Mid_sink')
snk_low = Sink('Low_sink')
snk_abgas = Sink('Abgas_sink')
# Splitter
splitter = Splitter('splitter', num_out=3)
# Compressors/Pumps
comp_high = Compressor('High_compressor')
comp_mid  = Compressor('Mid_compressor')
comp_low  = Compressor('Low_compressor')
# Economizers
eco_high = HeatExchanger('High_economizer')
eco_mid  = HeatExchanger('Mid_economizer')
eco_low  = HeatExchanger('Low_economizer')
# Heat Exchangers
wu_high = HeatExchanger('High_he')
wu_mid  = HeatExchanger('Mid_he')
wu_low  = HeatExchanger('Low_he')


# CONNECTIONS


c00 = Connection(src_water, 'out1', splitter, 'in1', label = '00')

c31 = Connection(splitter, 'out1', comp_high, 'in1', label = '31')
c41 = Connection(splitter, 'out2', comp_mid,  'in1', label = '41')
c51 = Connection(splitter, 'out3', comp_low,  'in1', label = '51')

c32 = Connection(comp_high, 'out1', eco_high, 'in2', label = '32')
c42 = Connection(comp_mid,  'out1', eco_mid,  'in2', label = '42')
c52 = Connection(comp_low,  'out1', eco_low,  'in2', label = '52')

c33 = Connection(eco_high, 'out2', wu_high,  'in2', label = '33')
c43 = Connection(eco_mid,  'out2', wu_mid,   'in2', label = '43')
c53 = Connection(eco_low,  'out2', wu_low,   'in2', label = '53')

c34 = Connection(wu_high,  'out2', snk_high, 'in1', label = '34')
c44 = Connection(wu_mid,   'out2', snk_mid,  'in1', label = '44')
c54 = Connection(wu_low,   'out2', snk_low,  'in1', label = '54')

c11 = Connection(src_abgas, 'out1', wu_high,   'in1', label = '11')
c12 = Connection(wu_high,   'out1', wu_mid,    'in1', label = '12')
c13 = Connection(wu_mid,    'out1', wu_low,    'in1', label = '13')
c14 = Connection(wu_low,    'out1', eco_high,  'in1', label = '14')
c15 = Connection(eco_high,  'out1', eco_mid,   'in1', label = '15')
c16 = Connection(eco_mid,   'out1', eco_low,   'in1', label = '16')
c17 = Connection(eco_low,   'out1', snk_abgas, 'in1', label = '17')


dampferzeuger.add_conns(c00, c11, c12, c13, c14, c15, c16, c17, c31, c32, c33, c34, c41, c42, c43, c44, c51, c52, c53, c54)


# PARAMETERS

#components
#splitter.set_attr(pr = 1)
comp_high.set_attr(eta_s = 0.8)
comp_mid.set_attr(eta_s = 0.8)
comp_low.set_attr(eta_s = 0.8)

wu_high.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20)
wu_mid.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20)
wu_low.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20)

eco_high.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20) #check les lowest differences
eco_mid.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20)  #check les lowest differences
eco_low.set_attr(pr1 = 1, pr2 = 1, ttd_l = -20)  #check les lowest differences


#connections

c00.set_attr(p = 1, T = 99.606, m = 1350, fluid = fld_wf)
c31.set_attr(m = 90)
c41.set_attr(m = 360)
#c51.set_attr(m = 900)
c32.set_attr(p = 41) # Adding 1 to the pressure because of future pressure losses
c42.set_attr(p = 15) # Adding 1 to the pressure because of future pressure losses
c52.set_attr(p = 5)  # Adding 1 to the pressure because of future pressure losses

c11.set_attr(p = 30, T = 2000, m = 6100, fluid = fld_abgas) # using random values to check amount of variables

motor_high = Bus('High_motor')
motor_high.add_comps({'comp' : comp_high, 'char' : 0.975, 'base' : 'bus'})
motor_mid = Bus('Mid_motor')
motor_mid.add_comps({'comp' : comp_mid, 'char' : 0.975, 'base' : 'bus'})
motor_low = Bus('Low_motor')
motor_low.add_comps({'comp' : comp_low, 'char' : 0.975, 'base' : 'bus'})

dampferzeuger.add_busses(motor_high, motor_mid, motor_low)

# SOLVE
dampferzeuger.solve(mode='design')
dampferzeuger.print_results()


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.83e+09   | 0 %        | 2.50e+02   | 9.52e+06   | 1.68e+12   | 0.00e+00   | 0.00e+00   
 2     | 3.22e+09   | 0 %        | 1.12e-13   | 3.16e+06   | 6.82e+08   | 0.00e+00   | 0.00e+00   
 3     | 2.11e+09   | 0 %        | 7.39e-14   | 1.06e+06   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 4     | 1.62e+09   | 0 %        | 8.48e-14   | 1.77e+05   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 5     | 1.58e+09   | 0 %        | 2.44e-18   | 2.41e-10   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 6     | 1.58e+09   | 0 %        | 1.92e-23   | 3.40e-10   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 7     | 1.58e+09   | 0 %        | 1.21e-28   | 2.41e-10   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 8     | 1.58e+09   | 0 %        | 9.88e-30   | 3.40e-10   | 5.71e+08   | 0.00e+00   | 0.00e+00   
 9     | 

Invalid value for ttd_u: ttd_u = -474.33204830900456 below minimum value (0) at component Low_economizer.
Invalid value for ttd_l: ttd_l = -20.00000000000057 below minimum value (0) at component Low_economizer.
Invalid value for ttd_u: ttd_u = -790.6216209065328 below minimum value (0) at component High_economizer.
Invalid value for ttd_l: ttd_l = -19.999999999999204 below minimum value (0) at component High_economizer.
Invalid value for ttd_l: ttd_l = -20.000000000000682 below minimum value (0) at component Low_he.
Invalid value for ttd_u: ttd_u = -1038.6540654530777 below minimum value (0) at component Mid_economizer.
Invalid value for ttd_l: ttd_l = -20.000000000001364 below minimum value (0) at component Mid_economizer.
The solver does not seem to make any progress, aborting calculation. Residual value is 1.58e+09. This frequently happens, if the solver pushes the fluid properties out of their feasible range.



##### RESULTS (Compressor) #####
+-----------------+----------+----------+----------+--------+
|                 |        P |    eta_s |       pr |   igva |
|-----------------+----------+----------+----------+--------|
| Mid_compressor  | 8.02e+07 | 8.00e-01 | 1.50e+01 |    nan |
| Low_compressor  | 1.04e+08 | 8.00e-01 | 5.00e+00 |    nan |
| High_compressor | 3.12e+07 | 8.00e-01 | 4.10e+01 |    nan |
+-----------------+----------+----------+----------+--------+
##### RESULTS (HeatExchanger) #####
+-----------------+-----------+------------+------------+-----------+-----------+----------+----------+----------+----------+
|                 |         Q |         kA |     td_log |     ttd_u |     ttd_l |      pr1 |      pr2 |    zeta1 |    zeta2 |
|-----------------+-----------+------------+------------+-----------+-----------+----------+----------+----------+----------|
| High_he         | -6.39e+08 |   3.43e+06 |   1.86e+02 |  2.00e+03 |  4.28e-02 | 1.00e+00 | 1.00e+00 | 0.00e+00 | 0.0

In [23]:
# Open Questions :
# How to use ttd_l and ttd_u
# How to modelise the pressure loss (pipe?)
# Impose pressure and flux at the sinks ?
# Add entgaser 
# Same component for economizers and wärmeübertrager
# Converging issues because properties out of range (both in combustion chamber and in Dampf part)
# 